# Data access and data cleaning

Data cleaning forms an important if messy part of preparing real-world data for use in a database or a data analysis process.  Cleaning is intended to standardize data formats and representations into a more legible form factor for the machine.

What sorts of things can go wrong when dealing with data?

- Data in incorrect format
- Inconsistent labels and values
- Poorly chosen empty values
- Gaps or overlaps in data
- Poor quality conversion (as in OCR text)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Data Description

We will utilize weather data for the Coleto Creek Reservoir from 2003–14, provided by the [National Oceanic and Atmospheric Administration (NOAA)](https://www.noaa.gov/).  The data set contains 4,197 daily weather observations of evaporation–transpiration, precipitation, and temperature extremes.

![](http://water.weather.gov/ahps2/images/hydrograph_photos/ckdt2/dsc00171.jpg)

The data set is organized as a comma-separated value (CSV) file with the fields `LATITUDE`, `LONGITUDE`, `DATE`, `ET`, `PRCP`, `TMAX`, `TMIN`.  Pandas provides a spreadsheet-like or database-like window on the data structure.  We will load the data with the Pandas function `read_csv`:

The data structure itself is what Pandas calls a `DataFrame`.  Again, think of a spreadsheet containing columns of data.  These columns are Pandas `Series` and can be accessed by their column header:

## Data Cleaning

Many of the observations included in this data set as raw data are incomplete—in other words, the raw data contain missing values and outliers.  These can be troublesome since the format of missing data can vary widely—for instance, `0`, `NaN`, `""`, `NULL`, and `-9999` all occur in practice—and so we commonly need to clean data by either removing entries, standardizing missing values to a useful default, or interpolating values (only if necessary and done carefully!).

In this tutorial, we will focus on `TMAX` and `TMIN` data to demonstrate how to clean data of missing values and outliers.  First, let's examine the distribution of `TMAX` values.

For this particular data set, missing values are represented by `-9999`.  We need to filter out these rows and handle the values in a more transparent way.  We could either replace the values with non-calculating values (like `NaN`), or remove the lines directly.  In this case, we will remove the lines directly, but I'll show you as well how the first approach works.

What else could need checking in this database for $T_\textrm{min}$ and $T_\textrm{max}$?  Let's make sure they are ordered properly.  (Note that we can access a column then a row, or a row then a column.)  Simply find and swap any cases where this occurs:

Oops, that gives us the persistent problem that DataFrames don't like to change their values.  Explicitly, if you _mutate_ (change) data, Pandas generates a copy.  Thus the `values` method at the end there.

Now plot the histogram of temperature data again:

Next, what's going on with the scale?  Well, it turns out that the values are stored in tenths of a degree centigrade, which allows them to be stored without a decimal point.  Let's fix that as well:

Given the correct range and cleaned data values, now examine the time series form:


-   Why are the values clustered apart from each other?  We'll need to convert that column (`DATE`) to a Python `datetime` representation:

Try again:

Also check the box plot distribution form:

## Basic Statistical Analysis

In this section, we will show you how to perform some basic statistical
analysis of the dataset.  This will help us diagnose any additional basic features.  As an example, select year 2008.

Locate the extremes---the hottest and the coldest day in the year.

We can also find the days in which the maximum temperature exceeds 35 °C.

## Data Smoothing

Data smoothing is an important way to see general patterns
associated with a dataset.  Noisy data tend to obscure the visibility of trends.  In this section, we will introduce several data smoothing functions available in Pandas.

Some of these rely on the concept of a _rolling window_, in which a sample of several neighboring values is used to determine the behavior at a given point in time.  Others are sampling filters used in signal processing applications.

First, let's examine the _rolling mean_ and _rolling standard deviation_.  These are built into Pandas.

-   Why are there gaps in the rolling mean?

-   How sensitive is the system to the value of `n`?

We can use these together to build a very nice profile plot:

Next, let's use some low-pass filtering to smooth the data.  (Low-pass filters only pass signal components that are below a threshold frequency.  This helps because noise tends to look like high-frequency variability.)  The Savitsky–Golay filter is popular:

Thirdly, we use a [forward–backward filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html) to smooth the data.  (This technique applies a linear filter once forward and once backward to remove offset.  Note that the rolling mean, for instance, tends to lag the signal in time series.)  Since we need to use a filtering algorithm with this, we'll introduce the [Butterworth](https://infogalatic.com/info/Butterworth_filter) [filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html#scipy.signal.butter), which is designed to be as "flat" as possible.

Finally, we can compare the original data with each of the smoothed data methods:

-   How can we quantify the "goodness" of these smooth sets?  [differentiability,residuals]

### Correlations Between Variables

Correlation can help us understand the relationship between different
variables.  For variables $x$ and $y$, correlation $\rho$ can be computered
as:

$$
\rho_{x,y}
=
\frac{E[x-\mu_x][y-\mu_y]}{\sigma_{x}\sigma_{y}}
$$

Let's see the correlation of maximum temperature, minimum temperature and E–T:

If you are willing to install some other packages, there are some nice visualizations available:

`TMAX` and `TMIN` are clearly correlated with each other, while they are not
correlated with `EVAP`.

Now we want to see if the `TMAX` of one day is correlated with the `TMAX` of other days.

Clearly `TMAX` values become less correlated as time lag increases.

## Contributors

These lessons were developed by Erhu Du, Jane Lee, and Neal Davis for Computational Science and Engineering at the University of Illinois.  Development was supported by a grant from MathWorks, Inc.